In [1]:
import pandas as pd
import os
import csv

import warnings
warnings.simplefilter("ignore")

`extraction_daterange`: Range de datas para o tratamento de dados

`input_extraction_date`: Texto com referência ao nome do diretório da última extração

`path_etl`: Caminho para o diretório de extrações, combinando variáveis anteriores

`path_export`: Caminho para o diretório da camada clean

In [6]:
extraction_daterange = "365d"
input_extraction_date = "04_23-04_24"

path_etl = os.path.join(os.getcwd(), "data", "raw", extraction_daterange, input_extraction_date)
path_export = os.path.join(os.getcwd(), "data", "processed", extraction_daterange)

etl_files = os.listdir(path_etl)
try:
    path_concorrentes = path_etl + "/" + [file for file in etl_files if "concorrentes" in file or "competitors" in file][0]
    path_conteudo = path_etl + "/" + [file for file in etl_files if "conteudo" in file or "content" in file or "conteúdo" in file][0] 
    path_seguidores = path_etl + "/" + [file for file in etl_files if "seguidores" in file or "followers" in file][0]
    path_visitantes = path_etl + "/" + [file for file in etl_files if "visitantes" in file or "visitors" in file][0]
except Exception as e:
    print("Erro ao encontrar arquivos de entrada. Verifique o diretório.")
    print(str(e))
    

`ultima_data_23`: Última data da primeira exportação (período de 1 ano)
`ultima_data_24`: Última data da última exportação

`concat_dfs`: Função para concatenar dataframes.

    Args: 
        df1, df2: Dataframes
        drop_duplicates: Recebe strings com nomes de colunas para remover duplicatas
    
`english_cols`: Dicionário com o mapeamento das categorias e colunas em inglês

`transform_data`: Realiza o tratamento de dados de um dataframe
    
    Args:
        df: Dataframe
        category: string de referência para o tratamento de dados

    Return:
        Dataframe

`get_last_date`: Retorna a última data do dataframe

    Args:
        df: dataframe

    Return:
        Date

In [8]:
# funções
def concat_dfs(
    df1,
    df2,
    drop_duplicates=False,
):
    df1["Date"] = pd.to_datetime(df1["Date"])
    df2["Date"] = pd.to_datetime(df2["Date"])
    df = pd.concat([df1, df2])

    if drop_duplicates:
        df = df.drop_duplicates(subset=[drop_duplicates])

    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df = df.sort_values(by="Date", ascending=False)

    return df


english_cols = {
    "content_metrics": [
        "Date",
        "Impressions (organic)",
        "Impressions (sponsored)",
        "Impressions",
        "Unique impressions (organic)",
        "Clicks (organic)",
        "Clicks (sponsored)",
        "Clicks",
        "Reactions (organic)",
        "Reactions (sponsored)",
        "Reactions",
        "Comments (organic)",
        "Comments (sponsored)",
        "Comments",
        "Shares (organic)",
        "Shares (sponsored)",
        "Shares",
        "Engagement rate (organic)",
        "Engagement rate (sponsored)",
        "Engagement rate",
    ],
    "content_posts": [
        "Post Title",
        "Post Link",
        "Post Type",
        "Campaign Name",
        "Published by",
        "Date",
        "Campaign Start Date",
        "Campaign End Date",
        "Audience",
        "Impressions",
        "Views (excluding off-site video views)",
        "Off-site Views",
        "Clicks",
        "Click-Through Rate (CTR)",
        "Likes",
        "Comments",
        "Shares",
        "Followers",
        "Engagement Rate",
        "Content Type",
    ],
    "followers_new": [
        "Date",
        "Followers Sponsored",
        "Followers Organic",
        "Total Followers",
    ],
    "followers_location": ["Location", "Total Followers"],
    "followers_function": ["Function", "Total Followers"],
    "followers_experience": ["Experience Level", "Total Followers"],
    "followers_industry": ["Industry", "Total Followers"],
    "followers_company_size": ["Company Size", "Total Followers"],
    "visitors_metrics": [
        "Date",
        "Page Views Overview (Desktop)",
        "Page Views Overview (Mobile Devices)",
        "Page Views Overview (Total)",
        "Unique Visitors Overview (Desktop)",
        "Unique Visitors Overview (Mobile Devices)",
        "Unique Visitors Overview (Total)",
        "Page Views Day by Day (Desktop)",
        "Page Views Day by Day (Mobile Devices)",
        "Page Views Day by Day (Total)",
        "Unique Visitors Day by Day (Desktop)",
        "Unique Visitors Day by Day (Mobile Devices)",
        "Unique Visitors Day by Day (Total)",
        "Page Views Jobs (Desktop)",
        "Page Views Jobs (Mobile Devices)",
        "Page Views Jobs (Total)",
        "Unique Visitors Jobs (Desktop)",
        "Unique Visitors Jobs (Mobile Devices)",
        "Unique Visitors Jobs (Total)",
        "Total Page Views (Desktop)",
        "Total Page Views (Mobile Devices)",
        "Total Page Views (Total)",
        "Total Unique Visitors (Desktop)",
        "Total Unique Visitors (Mobile Devices)",
        "Total Unique Visitors (Total)",
    ],
    "visitors_location": ["Location", "Total Views"],
    "visitors_function": ["Function", "Total Views"],
    "visitors_experience": ["Experience Level", "Total Views"],
    "visitors_industry": ["Industry", "Total Views"],
    "visitors_company_size": ["Company Size", "Total Views"],
    "competitors": [
        "Page",
        "Total Followers",
        "New Followers",
        "Total Post Engagements",
        "Total Posts",
    ],
}

last_date = None  # O valor é atribuido durante o loop de limpeza e utilizado em transform_data, caso a tabela não tenha a medida de tempo


def transform_data(df, category):
    df.columns = english_cols.get(category)

    if category == "content_metrics":
        df = df[
            [
                "Date",
                "Impressions",
                "Clicks",
                "Reactions",
                "Comments",
                "Shares",
                "Engagement rate",
            ]
        ]

        df["Reactions (positive)"] = df["Reactions"][df["Reactions"] >= 0]
        df["Comments (positive)"] = df["Comments"][df["Comments"] >= 0]
        df["Shares (positive)"] = df["Shares"][df["Shares"] >= 0]
        df["Clicks (positive)"] = df["Clicks"][df["Clicks"] >= 0]

        df["Reactions (positive)"] = df["Reactions (positive)"].fillna(0)
        df["Comments (positive)"] = df["Comments (positive)"].fillna(0)
        df["Shares (positive)"] = df["Shares (positive)"].fillna(0)
        df["Clicks (positive)"] = df["Clicks (positive)"].fillna(0)

        window = 3

        df["Reactions (moving average)"] = (
            df["Reactions (positive)"].rolling(window=window).mean()
        )
        df["Comments (moving average)"] = (
            df["Comments (positive)"].rolling(window=window).mean()
        )
        df["Shares (moving average)"] = (
            df["Shares (positive)"].rolling(window=window).mean()
        )
        df["Clicks (moving average)"] = (
            df["Clicks (positive)"].rolling(window=window).mean()
        )

        df["Reactions"] = df.apply(
            lambda row: (
                row["Reactions (moving average)"]
                if row["Reactions"] < 0
                else row["Reactions"]
            ),
            axis=1,
        )

        df["Comments"] = df.apply(
            lambda row: (
                row["Comments (moving average)"]
                if row["Comments"] < 0
                else row["Comments"]
            ),
            axis=1,
        )

        df["Shares"] = df.apply(
            lambda row: (
                row["Shares (moving average)"] if row["Shares"] < 0 else row["Shares"]
            ),
            axis=1,
        )

        df["Clicks"] = df.apply(
            lambda row: (
                row["Clicks (moving average)"] if row["Clicks"] < 0 else row["Clicks"]
            ),
            axis=1,
        )

        df["Engagement Rate"] = df.apply(
            lambda row: (
                row["Reactions"] + row["Comments"] + row["Clicks"] + row["Shares"]
            )
            / row["Impressions"],
            axis=1,
        )

        df = df[
            [
                "Date",
                "Impressions",
                "Clicks",
                "Reactions",
                "Comments",
                "Shares",
                "Engagement Rate",
            ]
        ]

    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])

    else:
        df.insert(0, "Date", last_date)

    return df


def get_last_date(df):
    return df["Date"].max().date()

### Leitura, Tratamento, Concatenação e Exportação para CSV

In [9]:
tables = [
    {"name": "content_metrics", "path": path_conteudo, "sheet": 0, "skiprows": 1},
    {"name": "content_posts", "path": path_conteudo, "sheet": 1, "skiprows": 1},
    {"name": "followers_new", "path": path_seguidores, "sheet": 0},
    {"name": "followers_location", "path": path_seguidores, "sheet": 1},
    {"name": "followers_function", "path": path_seguidores, "sheet": 2},
    {"name": "followers_experience", "path": path_seguidores, "sheet": 3},
    {"name": "followers_industry", "path": path_seguidores, "sheet": 4},
    {"name": "followers_company_size", "path": path_seguidores, "sheet": 5},
    {"name": "followers_company_size", "path": path_seguidores, "sheet": 5},
    {"name": "visitors_metrics", "path": path_visitantes, "sheet": 0},
    {"name": "visitors_location", "path": path_visitantes, "sheet": 1},
    {"name": "visitors_function", "path": path_visitantes, "sheet": 2},
    {"name": "visitors_experience", "path": path_visitantes, "sheet": 3},
    {"name": "visitors_industry", "path": path_visitantes, "sheet": 4},
    {"name": "visitors_company_size", "path": path_visitantes, "sheet": 5},
    {"name": "competitors", "path": path_concorrentes, "skiprows": 1},
]

# read
print("Reading data...")
for table in tables:
    if ".xls" in table["path"]:
        table["df"] = pd.read_excel(
            table["path"],
            sheet_name=table.get("sheet", 0),
            skiprows=table.get("skiprows", 0),
        )
    else:
        table["df"] = pd.read_csv(table["path"], skiprows=table.get("skiprows", 0))


# transform
print("Transforming data...")
for table in tables:
    if table["name"] == "competitors":
        table["df"] = table["df"][
            [
                "Page",
                "Total de seguidores",
                "Novos seguidores",
                "Total de engajamentos da publicação",
                "Total de publicações",
            ]
        ]

    table["df"] = transform_data(table["df"], category=table["name"])
    if table["name"] == "content_metrics":
        last_date = get_last_date(table["df"])
        print("*Ultima data coletada* -", last_date)


# concatenating if has clean data
if len(os.listdir(path_export)) > 0:
    print("Clean data detected! Concatenating...")
    for table in tables:
        df_clean = pd.read_csv(path_export + "\\" + table["name"] + ".csv", parse_dates=['Date'])
        
        if table["name"] == "content_metrics":
            table["df"] = concat_dfs(df_clean, table["df"], drop_duplicates="Date")
        elif table["name"] == "content_posts":
            table["df"] = concat_dfs(df_clean, table["df"], drop_duplicates="Post Link")
        else:
            table["df"] = concat_dfs(df_clean, table["df"])

# exporting
print("Exporting data...")
for table in tables:
    filepath = path_export + f"/{table['name']}.csv"
    table["df"].to_csv(filepath, index=False, quoting=csv.QUOTE_ALL)


print("Done!")

Reading data...
Transforming data...
*Ultima data coletada* - 2024-03-31
Clean data detected! Concatenating
Exporting data...
Done!


## V2 - Tests

In [ ]:
# concatenating 

if len(os.listdir(path_export)) > 0:
    for table in tables:
        df_clean = pd.read_csv(path_export + "\\" + table["name"] + ".csv", parse_dates=['Date'])
        
        if table["name"] == "content_metrics":
            table["df"] = concat_dfs(df_clean, table["df"], drop_duplicates="Date")
        elif table["name"] == "content_posts":
            table["df"] = concat_dfs(df_clean, table["df"], drop_duplicates="Post Link")
        else:
            table["df"] = concat_dfs(df_clean, table["df"])

tables[0]["df"]

In [ ]:
# etl conteudo

df_content_metrics = transform_data(
    pd.read_excel(path_conteudo, sheet_name=0, skiprows=1), "content_metrics"

)

df_content_posts = transform_data(
    pd.read_excel(path_conteudo, sheet_name=1, skiprows=1), "content_posts"

)



# etl seguidores


df_followers_new = transform_data(
    pd.read_excel(path_seguidores, sheet_name=0), "followers_new"

)


df_followers_location = transform_data(
    pd.read_excel(path_seguidores, sheet_name=1), "followers_location"

)


df_followers_function = transform_data(
    pd.read_excel(path_seguidores, sheet_name=2), "followers_function"

)



df_followers_experience = transform_data(
    pd.read_excel(path_seguidores, sheet_name=3), "followers_experience"

)



df_followers_industry = transform_data(
    pd.read_excel(path_seguidores, sheet_name=4), "followers_industry"

)



df_followers_company_size = transform_data(
    pd.read_excel(path_seguidores, sheet_name=5), "followers_company_size"

)



# visitantes


df_visitors_metrics = transform_data(
    pd.read_excel(path_visitantes, sheet_name=0), "visitors_metrics"

)



df_visitors_location = transform_data(
    pd.read_excel(path_seguidores, sheet_name=1), "visitors_location"

)



df_visitors_function = transform_data(
    pd.read_excel(path_seguidores, sheet_name=2), "visitors_function"

)



df_visitors_experience = transform_data(
    pd.read_excel(path_seguidores, sheet_name=3), "visitors_experience"

)



df_visitors_industry = transform_data(
    pd.read_excel(path_seguidores, sheet_name=4), "visitors_industry"

)



df_visitors_company_size = transform_data(
    pd.read_excel(path_seguidores, sheet_name=5), "visitors_company_size"

)



# concorrentes
default_columns = [
    "Page",
    "Total de seguidores",
    "Novos seguidores",
    "Total de engajamentos da publicação",
    "Total de publicações",
]

df_concorrentes = (
    pd.read_csv(path_concorrentes, skiprows=1)[default_columns]
    if "csv" in path_concorrentes
    else pd.read_excel(path_concorrentes, skiprows=1)[default_columns]
)

df_competitors = transform_data(df_concorrentes, category="competitors")

In [ ]:
df_seguidores_tamanho_empresa

## V1

In [ ]:
# conteudo
df_conteudo_metricas = concat_dfs(
    transform_data(
        pd.read_excel(path_conteudo_23, sheet_name=0, skiprows=1), "content_metrics"
    ),
    transform_data(
        pd.read_excel(path_conteudo_24, sheet_name=0, skiprows=1), "content_metrics"
    ),
)

df_conteudo_publicacoes = concat_dfs(
    transform_data(
        pd.read_excel(path_conteudo_23, sheet_name=1, skiprows=1), "content_posts"
    ),
    transform_data(
        pd.read_excel(path_conteudo_24, sheet_name=1, skiprows=1), "content_posts"
    ),
    drop_duplicates="Post Link",
)

df_conteudo_metricas.to_csv(
    path_export + "/conteudo_metricas.csv", index=False, quoting=csv.QUOTE_ALL
)
df_conteudo_publicacoes.to_csv(
    path_export + "/conteudo_publicacoes.csv", index=False, quoting=csv.QUOTE_ALL
)

In [ ]:
# seguidores
df_seguidores_novos = concat_dfs(
    transform_data(pd.read_excel(path_seguidores_23, sheet_name=0), "followers_new"),
    transform_data(pd.read_excel(path_seguidores_24, sheet_name=0), "followers_new"),
    "Date",
)

df_seguidores_localidade = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=1), "followers_location"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=1), "followers_location"
    ),
)

df_seguidores_funcao = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=2), "followers_function"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=2), "followers_function"
    ),
)

df_seguidores_experiencia = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=3), "followers_experience"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=3), "followers_experience"
    ),
)

df_seguidores_setor = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=4), "followers_industry"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=4), "followers_industry"
    ),
)

df_seguidores_tamanho_empresa = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=5), "followers_company_size"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=5), "followers_company_size"
    ),
)

df_seguidores_novos.to_csv(
    path_export + "/seguidores_novos.csv", index=False, quoting=csv.QUOTE_ALL
)
df_seguidores_localidade.to_csv(
    path_export + "/seguidores_localidade.csv", index=False, quoting=csv.QUOTE_ALL
)
df_seguidores_funcao.to_csv(
    path_export + "/seguidores_funcao.csv", index=False, quoting=csv.QUOTE_ALL
)
df_seguidores_experiencia.to_csv(
    path_export + "/seguidores_experiencia.csv", index=False, quoting=csv.QUOTE_ALL
)
df_seguidores_setor.to_csv(
    path_export + "/seguidores_setor.csv", index=False, quoting=csv.QUOTE_ALL
)
df_seguidores_tamanho_empresa.to_csv(
    path_export + "/seguidores_tamanho_empresa.csv", index=False, quoting=csv.QUOTE_ALL
)

In [ ]:
# visitantes
df_visitantes_metricas = concat_dfs(
    transform_data(pd.read_excel(path_visitantes_23, sheet_name=0), "visitors_metrics"),
    transform_data(pd.read_excel(path_visitantes_24, sheet_name=0), "visitors_metrics"),
    "Date",
)

df_visitantes_localidade = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=1), "visitors_location"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=1), "visitors_location"
    ),
)

df_visitantes_funcao = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=2), "visitors_function"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=2), "visitors_function"
    ),
)

df_visitantes_experiencia = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=3), "visitors_experience"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=3), "visitors_experience"
    ),
)

df_visitantes_setor = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=4), "visitors_industry"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=4), "visitors_industry"
    ),
)

df_visitantes_tamanho_empresa = concat_dfs(
    transform_data(
        pd.read_excel(path_seguidores_23, sheet_name=5), "visitors_company_size"
    ),
    transform_data(
        pd.read_excel(path_seguidores_24, sheet_name=5), "visitors_company_size"
    ),
)

df_visitantes_metricas.to_csv(
    path_export + "/visitantes_metricas.csv", index=False, quoting=csv.QUOTE_ALL
)

df_visitantes_localidade.to_csv(
    path_export + "/visitantes_localidade.csv", index=False, quoting=csv.QUOTE_ALL
)

df_visitantes_funcao.to_csv(
    path_export + "/visitantes_funcao.csv", index=False, quoting=csv.QUOTE_ALL
)

df_visitantes_experiencia.to_csv(
    path_export + "/visitantes_experiencia.csv", index=False, quoting=csv.QUOTE_ALL
)

df_visitantes_setor.to_csv(
    path_export + "/visitantes_setor.csv", index=False, quoting=csv.QUOTE_ALL
)

df_visitantes_tamanho_empresa.to_csv(
    path_export + "/visitantes_tamanho_empresa.csv", index=False, quoting=csv.QUOTE_ALL
)

In [ ]:
# concorrentes
competitors_default_columns = [
    "Page",
    "Total de seguidores",
    "Novos seguidores",
    "Total de engajamentos da publicação",
    "Total de publicações",
]

df_concorrentes = (
    pd.read_csv(path_concorrentes, skiprows=1)[default_columns]
    if "csv" in path_concorrentes
    else pd.read_excel(path_concorrentes, skiprows=1)[default_columns]
)

df_concorrentes = transform_data(df_concorrentes, category="competitors")

df_concorrentes